In [1]:
from prody import *
import numpy as np
import csv
import pandas as pd
import sys, os
from glob import glob
# Add the main the parent directory
sys.path.append(r'..')

In [2]:
# Protein name
prot_name = 'fxa'
# Protein sequence: P28482 (ERK2_HUMAN)
uniprot_id = "P00742"

In [3]:

# Open the protein data table
path_df_csv = glob(os.path.join('..', 'data', 
                        F'TABLA_MTDATA_{prot_name.upper()}_*_crys.json'))
df_prot = pd.read_json(path_df_csv[0])
df_prot.set_index('PDB_ID', inplace=True)


In [4]:
# How many structures have a HETATM?
df_prot_ligs = df_prot[df_prot.NumLigs > 0]
print(F'{len(df_prot_ligs)} crystals {len(df_prot)} have at least one HETATM')

# How many ligands are there?
import itertools
# List_ name all ligands
lig_full_list = list( itertools.chain.from_iterable( df_prot.NameLigs ) )
print(F'There are {len(lig_full_list)} HETATM molecules/atoms')

lig_all_list = list(set(lig_full_list))
print(F'There are {len(lig_all_list)} DIFFERENT molecules/atoms.')

134 crystals 136 have at least one HETATM
There are 314 HETATM molecules/atoms
There are 147 DIFFERENT molecules/atoms.


## Moléculas en la cavidad catalítica
Ahora procedemos a identificar qué moléculas se encuentran en el sitio activo de la proteína, y si éstas tienen un peso molecular suficcientemente grande como para ser diferenciadas de posibles moléculas del solvente.

### Estructura de referencia
Tomamos como referencia alguna estructura del DataFrame con ligandos:

De acuerdo a BindingDatabase, el mejor ligando (y la estructura con él es: http://www.rcsb.org/pdb/results/results.do?tabtoshow=Current&qrid=EA4EE133

2P3T y ligando 993

In [6]:
from modules.get_cocristalized_ligands import PocketResidues

In [7]:
prot_main_dir = '../../FILES/CRYSTALS/PROT_FXA_PREPARED/PREP_ALL_ENSAMBLE'
raw_ligs_dir = '../../FILES/CRYSTALS/LIGS_FXA/RAW'

pdb_id = '1ezq'
ligand_resname = 'RPR'

ref_structure = PocketResidues(pdb_id, ligand_resname, prot_main_dir, raw_ligs_dir)
# Get the pocket redidues
pocket_residues = ref_structure.get_pocket_residues_as_str(cutoff=5)
pocket_residues

@> 1906 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 144 atoms and 1 coordinate set(s) were parsed in 0.01s.


'131 134 161 178 179 180 181 184 202 203 204 205 206 207 208 209 82 83 84 214 215 216'

In [8]:
# Visualize the pocket residues
view = ref_structure.visualize_pocket(cutoff=5)
view

NGLWidget()

Cualquier molécula que tenga un átomo en contacto con la esfera, será considerada como ligando unido al sitio activo.

## Determinación de los ligandos
Ahora, procedemos a utilizar la estructura de referencia modelada con Modeller. Hay que recordar que todas las estructuras modeladas fueron alineadas previamente tomando en cuenta la estructura secundaria de la proteína, y que el alineamiento incluyó la reposición relativa de los ligandos.


### Pasos y función para obtener los ligandos

In [9]:
# Definimos los directorios de trabajo
main_dir = '../../FILES/CRYSTALS'
RAW_LIG_DIR = F'{main_dir}/LIGS_{prot_name.upper()}/RAW/'
PROT_CHAINS_DIR = F'{main_dir}/PROT_{prot_name.upper()}_PREPARED/PREP_ALL_ENSAMBLE/'

# Directorio de salida (lo creamos si no existe):
PK_LIGS_DIR = F'{main_dir}/LIGS_{prot_name.upper()}/POCKET_LIGS/'
import os
if not os.path.exists(PK_LIGS_DIR):
    os.makedirs(PK_LIGS_DIR)

In [10]:
from modules.get_cocristalized_ligands import get_pocket_ligand

# Se agregan dos columnas al dataframe
df_prot["Inhib"] = ""
df_prot["Inhib_mass"] = np.nan

# Se obtienen los ligandos
for pdb_id in df_prot.index:
    inhibidor = get_pocket_ligand(pdb_id, pocket_residues=pocket_residues,
                                  raw_lig_dir= RAW_LIG_DIR,
                                  prot_chain_dir=PROT_CHAINS_DIR,
                                  pk_ligs_dir=PK_LIGS_DIR,
                                  write_files=True, cutoff=3.5, min_weight = 97)
    
    df_prot.at[str(pdb_id), 'Inhib'] = inhibidor[0]
    df_prot.at[pdb_id, 'Inhib_mass'] = inhibidor[1]

@> 1338 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 3781 atoms and 1 coordinate set(s) were parsed in 0.06s.
@> 144 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 1906 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 202 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 1906 atoms and 1 coordinate set(s) were parsed in 0.02s.


['HOH']
La proteina 1c5m NO TIENE LIGANDO.
['HOH' 'RPR']
Proteína 1ezq: ligando RPR guardado.
['815' 'HOH']
Proteína 1f0r: ligando 815 guardado.


@> 177 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 1906 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 40 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1906 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 239 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1906 atoms and 1 coordinate set(s) were parsed in 0.03s.


['HOH' 'PR2']
Proteína 1f0s: ligando PR2 guardado.
['DX9']
Proteína 1fax: ligando DX9 guardado.
['HOH' 'Z34']


@> 253 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1906 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 33 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 1906 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 207 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 1906 atoms and 1 coordinate set(s) were parsed in 0.02s.


...

['4O6' 'HOH' 'MG']
Proteína 4y79: ligando 4O6 guardado.
['987' 'HOH' 'MG']
Proteína 4y7a: ligando 987 guardado.
['44I' 'HOH']


@> 308 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1906 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 299 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 1906 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 183 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 1906 atoms and 1 coordinate set(s) were parsed in 0.02s.


Proteína 4y7b: ligando 44I guardado.
['4O4' 'HOH']
Proteína 4zh8: ligando 4O4 guardado.
['4O5' 'HOH' 'MG']
Proteína 4zha: ligando 4O5 guardado.
['6PK' 'CA' 'HOH']


@> 543 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 1906 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> 506 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1906 atoms and 1 coordinate set(s) were parsed in 0.02s.


Proteína 5k0h: ligando 6PK guardado.
['HOH' 'NA']
La proteina 5voe NO TIENE LIGANDO.
['HOH' 'NA' 'RIV']
Proteína 5vof: ligando RIV guardado.


## Updated Dataframe with ligands information

1. **Inhib:** Con el nombre de la molécula hayada en el sitio activo (código del PDB), si la hubo.
2. **Inhib_mass**: Con la masa del ligando.
    
Por último guardamos dicho DataFrame:

In [11]:
n_confs = df_prot.shape[0]
df_prot.to_json(F'./TABLA_MTDATA_{prot_name.upper()}_{n_confs}_crys_LIGS_INFO.json')